In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy.optimize as optimize
import scipy.integrate as integrate
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Loading the Data

In [3]:
files = ("/Users/ankitkothari/Downloads/yahoo_news.txt")

In [4]:
import numpy as np
import fileinput

def read_data(filenames):
    """
    Reads a stream of events from the list of given files.
    
    Parameters
    ----------
    filenames : list
        List of filenames
    
    Stores
    -------    
    articles : [article_ids]
    features : [[article_1_features] .. [article_n_features]]
    events : [
                 0 : displayed_article_index (relative to the pool),
                 1 : user_click,
                 2 : [user_features],
                 3 : [pool_indexes]
             ]
    """

    global articles, features, events, n_arms, n_events
    articles = []
    features = []
    events = []

    skipped = 0

    with fileinput.input(files=filenames) as f:
        for line in f:
            cols = line.split()
            if (len(cols) - 10) % 7 != 0:
                skipped += 1
            else:
                pool_idx = []
                pool_ids = []
                for i in range(10, len(cols) - 6, 7):
                    id = cols[i][1:]
                    if id not in articles:
                        articles.append(id)
                        features.append([float(x[2:]) for x in cols[i + 1: i + 7]])
                    pool_idx.append(articles.index(id))
                    pool_ids.append(id)

                events.append(
                    [
                        pool_ids.index(cols[1]),
                        int(cols[2]),
                        [float(x[2:]) for x in cols[4:10]],
                        pool_idx,
                    ]
                )
    features = np.array(features)
    n_arms = len(articles)
    n_events = len(events)
    print(n_events, "events with", n_arms, "articles")
    if skipped != 0:
        print("Skipped events:", skipped)

In [5]:
#about the data
print("Reading data...")
read_data(files)
print("Done.")

Reading data...
100000 events with 21 articles
Done.


In [6]:
print("Number of events:", n_events)
#n_arms is analogus to chosing the number of arms in the bandit problem (The number of articles in the pool) like the number of arms in the bandit problem
print("Number of articles:", n_arms)
print("Number of features:", features.shape[1])
print("Article Shape:", len(articles))
print("Event Shape:", len(events))

Number of events: 100000
Number of articles: 21
Number of features: 6
Article Shape: 21
Event Shape: 100000


In [7]:
#pool_idx is the index of the article in the pool of articles.
event_df = pd.DataFrame(events, columns=["article_idx", "user_click", "user_features", "pool_idx"])
event_df.head(2)

,article_idx,user_click,user_features,pool_idx
0,17,0,"[1.2e-05, 0.0, 6e-06, 2.3e-05, 0.999958, 1.0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]"
1,10,0,"[0.088932, 0.003915, 0.592269, 0.314084, 0.000801, 1.0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]"


In [8]:
ctr_by_article = event_df.groupby(["article_idx",'user_click']).size()
ctr_by_article.head(4)

article_idx  user_click
0            0             4963
             1              103
1            0             4776
             1              136
dtype: int64

In [9]:
ctr_by_article = ctr_by_article.unstack(level=1).reset_index()
ctr_by_article.rename(columns={0:'impressions',1:'clicks'},inplace=True)
ctr_by_article['ctr_observed'] = ctr_by_article['clicks']/ctr_by_article['impressions']
ctr_by_article

user_click,article_idx,impressions,clicks,ctr_observed
0,0,4963,103,0.020754
1,1,4776,136,0.028476
2,2,4962,173,0.034865
3,3,4882,168,0.034412
4,4,4829,116,0.024022
5,5,4771,183,0.038357
6,6,4830,127,0.026294
7,7,4996,98,0.019616
8,8,4821,74,0.015350
9,9,4744,114,0.024030


In [10]:
features_df = pd.DataFrame(features, columns=["article_feature_" + str(i) for i in range(features.shape[1])])
features_df.head(2)

,article_feature_0,article_feature_1,article_feature_2,article_feature_3,article_feature_4,article_feature_5
0,0.306008,0.00045,0.077048,0.230439,0.386055,1.0
1,0.306008,0.00045,0.077048,0.230439,0.386055,1.0


In [11]:
articles_df = pd.DataFrame(articles, columns=["article_id"])
articles_df.head(2)

,article_id
0,109498
1,109509


## Contextual multi-arm bandits
- The contextual bandit problem is a generalization of the multi-armed bandit that extends the model by making actions conditional on the state of the environment. Unlike the classical multi-armed bandit, it addresses the problem of identifying the most appropriate content given all relevant contextual signals.

    - Player gets context xt .
    - Player selects at ∈ A .
    - Player gets reward Rt(at) .
    - Given the history of actions, rewards and contexts Player updates their strategy.


- Given a set of presented articles, a reward is determined by the click-through behavior of the user. If user clicks on the article, a payout of 1 is incurred and 0 otherwise. Click-through-rate (CRT) is used to determine the selection and placement of ads within the recommendation application.

### Traing a conteextual multi-arm bandits

- In the learning phase, the algorithm,
    - Step 1 it selects an action and 
    - Step 2 obtains a reward, 
    - Step 3 updates its estimates of expected rewards of actions given the context.
    
_*This is usually done via feeding a single example consisting of x,a,r (context, action, reward) to an estimator model doing a single update step (batch size 1).*_

- The learn phase is usually common for most algorithms and the main differences come from the exploration phase. Different Exploring-Exploitation Strategies could be
    - Epsilon Greedy
    - UCB
    - Thomson Sampling

### About the Dataset

Dataset: R6A - Yahoo! Front Page Today Module User Click Log Dataset, version 1.0 (1.1 GB)
- Selected dataset contains a fraction of user click log for news articles displayed in the Featured Tab of the Today Module on Yahoo! Front Page (http://www.yahoo.com) during the first ten days in May 2009. The articles were chosen uniformly at random from a hand-picked pool of high-quality articles, which allows one to use a recently developed method to obtain an unbiased evaluation of an arbitrary bandit algorithm.



- Columns

  * timestamp: e.g., 1241160900
  * displayed_article_id: e.g., 109513
  * user_click (0 for no-click and 1 for click): e.g., 0
  * strings "|user" and "|{article_id}" indicate the start of user
  and article features
  * features are encoded as "feature_id:feature_value" pairs, and feature_id starts from 1.

- The pool of available articles for recommendation for each user visit is the set of articles that appear in that line of data. All user IDs (specifically, bcookies) are replaced by a common string 'user' so that no user information can be identified from this data.

- Each user or article is associated with six features. Feature #1 is the constant (always 1) feature, and features #2-6 correspond to the 5 membership features constructed via conjoint analysis with a bilinear model.



### Epsilon Greedy

- A simple ϵ-greedy method. Here ϵ defines the amount of exploration we wish to perform, and for the remaining 1-ϵ times the model prefers exploitation. By convention, “epsilon” represents the percentage of time/trials dedicated for exploration, and it is also typical to do random exploration. This introduces some form of stochasticity.
- The choose_arm function returns the best arm's index based on the ϵ-greedy policy. The update functiom updates algorithm's parameters(matrices) for the selected arm. It looks at the selected arm, updates the number of times this arm was chosen, and then it updates the mean reward observed for the selected arm.

- In explore phase the algorithm, given its internal estimates of goodness of each actions chooses the best possible action modulo exploration. For example, epsilon-greedy chooses the best action with probability (1−ϵ) and with probability ϵ it selects uniformly one of all possible actions.

In [12]:
class Egreedy:
    """
    Epsilon greedy algorithm implementation
    """

    def __init__(self, epsilon):
        """
        Parameters
        ----------
        epsilon : number (Egreedy parameter, ideally between 0 and 1)
        """

        self.e = round(epsilon, 1)  # epsilon parameter for Egreedy 
        self.algorithm = "Egreedy (ε=" + str(self.e) + ")"
        #n_arms is analogus to chosing the number of arms in the bandit problem (The number of articles in the pool) like the number of arms in the bandit problem
        self.q = np.zeros(n_arms)  # average reward for each arm -- this represents the known mean reward for each arm
        self.n = np.zeros(n_arms)  # number of times each article was chosen by the user

    def choose_arm(self, t, user, pool_idx):
        """
        Returns the best arm's index relative to the pool
        Parameters
        ----------
        t : number (number of trial)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        """

        p = np.random.rand()
        # This is where we are performing exploit-explore
        # the higher value of epsilon will favor on exploration
        if p > self.e: # we are exploiting the best action  
            max_index = np.argmax(self.q[pool_idx])  
            #print(f'Exploit: {max_index}, article: {articles[pool_idx[max_index]]}')    
            return max_index  #we return the article which has the highest average reward
        else: # exploring all the possible actions uniformly here
            return np.random.randint(low=0, high=len(pool_idx)) #we return a random article from the pool

    def update(self, displayed, reward, user, pool_idx):
        """
        Updates algorithm's parameters(matrices)
        Parameters
        ----------
        displayed : index (displayed article index relative to the pool)
        reward : binary (user clicked or not)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        """
        #index of the article in the pool of articles that was displayed
        a = pool_idx[displayed]
        
        # update counts pulled for chosen article 
        self.n[a] += 1
        
        # update average/mean value/reward for chosen arm
        self.q[a] += (reward - self.q[a]) / self.n[a]
        """
        this can also be written as:
        value = self.q[a]
        new_value = ((self.n[a]-1)/float(self.n[a])) * value + (1 / float(self.n[a])) * reward
        self.q[a] = new_value
        """

In [13]:
import random
import time

def evaluate(A, size=100, learn_ratio = 0.9):
    """
    Policy evaluator as described in the paper
    Parameters
    ----------
    A : class (algorithm)
    size : number (run the evaluation only on a portion of the dataset)
    learn_ratio : number (perform learning(update parameters) only on a small portion of the traffic)
    Returns
    -------
    learn : array (contains the ctr for each trial for the learning bucket)
    deploy : array (contains the ctr for each trial for the deployment bucket)
    """
    
    start = time.time()
    # we initialize the payoff and events parameters separately for learning phase of the events and deployment phase of events.
    Payoff_deploy = 0 # total payoff for the deployment bucket
    Payoff_learn = 0  # total payoff for the learning bucket
    Events_deploy = 1 # counter of valid events for the deployment bucket
    Events_learn = 0  # counter of valid events for the learning bucket

    learn = []
    deploy = []
    global events
    if size != 100:
        k = int(n_events * size / 100)
        events = random.sample(events, k)

    """
    we run through the logged events, and treat each event either for learning & updating the parameters,
    or for deployment purposes wherein we use the reward obtained as evaluation metric
    """
    zzz = 0
    for t, event in enumerate(events):

        displayed = event[0] #index of the article in the pool of articles that was displayed to the user
        reward = event[1] #binary (user clicked or not)
        user = event[2] #array (user features)
        pool_idx = event[3] #array of indexes (pool indexes for article identification)

        # select the arm based on the bandit policy

        # Payoff_learn + Payoff_deploy -> Sums upto number of times the arms are pulled
        # this simultes the no. of trials in our case
        chosen = A.choose_arm(Payoff_learn + Payoff_deploy, user, pool_idx)
        
        """
        If, given the current history ht−1, it happens that the policy A chooses the same arm a
        as the one that was selected by the logging policy, then the event is retained
        (that is, added to the history), and the total payoff updated.
        Otherwise, if the policy A selects a different arm from the one that was taken by the logging policy,
        then the event is entirely ignored, and the algorithm proceeds to the next event without any change in its state.
        """
        if chosen == displayed: # if the chosen arm is the same as the displayed arm
            if random.random() < learn_ratio: # if the event is within the learning bucket
                Payoff_learn += event[1] # update the total payoff for the learning bucket
                Events_learn += 1 # update the total number of valid events for the learning bucket
                A.update(displayed, reward, user, pool_idx) # update the parameters of the algorithm for the learning bucket 
                learn.append(Payoff_learn / Events_learn) # update the ctr for the learning bucket
            else:
                Payoff_deploy += event[1] # update the total payoff for the deployment bucket
                Events_deploy += 1 # update the total number of valid events for the deployment bucket
                deploy.append(Payoff_deploy / Events_deploy) # update the ctr for the deployment bucket

    end = time.time()

    execution_time = round(end - start, 1)
    execution_time = (
        str(round(execution_time / 60, 1)) + "m"
        if execution_time > 60
        else str(execution_time) + "s"
    )
    print(
        "{:<20}{:<10}{}".format(
            A.algorithm, round(Payoff_deploy / Events_deploy, 4), execution_time
        )
    )
    return learn, deploy

In [14]:
_, deploy = evaluate(Egreedy(0.1),learn_ratio=0.25)
rnd_ctr = deploy[-1]

_, deploy = evaluate(Egreedy(0.1),learn_ratio=0.5)
rnd_ctr = deploy[-1]

_, deploy = evaluate(Egreedy(0.1),learn_ratio=0.9)
rnd_ctr = deploy[-1]

_, deploy = evaluate(Egreedy(0.25),learn_ratio=0.5)
rnd_ctr = deploy[-1]

_, deploy = evaluate(Egreedy(0.5),learn_ratio=0.5)
rnd_ctr = deploy[-1]

_, deploy = evaluate(Egreedy(0.75),learn_ratio=0.5)
rnd_ctr = deploy[-1]

Egreedy (ε=0.1)     0.0288    0.7s
Egreedy (ε=0.1)     0.0295    0.8s
Egreedy (ε=0.1)     0.0202    0.8s
Egreedy (ε=0.2)     0.031     0.8s
Egreedy (ε=0.5)     0.0353    0.8s
Egreedy (ε=0.8)     0.0279    0.6s


In [42]:
best_epsilon = 0.2
eps_algo = Egreedy(best_epsilon)
_, deploy = evaluate(eps_algo, learn_ratio=0.5)

Egreedy (ε=0.2)     0.0333    0.6s


In [43]:
def epsilon_analysis(modeled_algo_instance, ctr_by_article):
    ctr_modeled = modeled_algo_instance.q
    article_displayed = modeled_algo_instance.n
    epsilon_algo_data = pd.DataFrame({"article_displayed_cnt": article_displayed, "ctr_modeled": ctr_modeled}).sort_values(by="ctr_modeled", ascending=False).reset_index()
    epsilon_algo_data.rename(columns={"index": "article_idx"}, inplace=True)
    ctr_by_article = ctr_by_article.merge(epsilon_algo_data, on="article_idx", suffixes=("_logged", "_modeled"))
    print(f'avg ctr_logged: {ctr_by_article["ctr_observed"].mean()}')
    print(f'avg ctr_modeled: {ctr_by_article["ctr_modeled"].mean()}')
    return ctr_by_article

In [44]:
epsilon_analysis(eps_algo, ctr_by_article)

avg ctr_logged: 0.029106945409521717
avg ctr_modeled: 0.012373867335518783


,article_idx,impressions,clicks,ctr_observed,article_displayed_cnt,ctr_modeled
0,0,4963,103,0.020754,57.0,0.017544
1,1,4776,136,0.028476,28.0,0.000000
2,2,4962,173,0.034865,37.0,0.027027
3,3,4882,168,0.034412,21.0,0.000000
4,4,4829,116,0.024022,39.0,0.025641
5,5,4771,183,0.038357,17.0,0.000000
6,6,4830,127,0.026294,28.0,0.000000
7,7,4996,98,0.019616,21.0,0.000000
8,8,4821,74,0.015350,36.0,0.027778
9,9,4744,114,0.024030,1921.0,0.033837


In [ ]:
ctr_modeled = eps_algo.q
article_displayed = eps_algo.n
epsilon_algo_data = pd.DataFrame({"article_displayed_cnt": article_displayed, "ctr_modeled": ctr_modeled}).sort_values(by="ctr_modeled", ascending=False).reset_index()
epsilon_algo_data.rename(columns={"index": "article_idx"}, inplace=True)

In [ ]:
epsilon_algo_data.head()

,article_idx,article_displayed_cnt,ctr_modeled
0,12,113.0,0.061947
1,14,24.0,0.041667
2,4,1792.0,0.039621
3,9,27.0,0.037037
4,3,30.0,0.033333


In [ ]:
ctr_by_article = ctr_by_article.merge(epsilon_algo_data, on="article_idx", suffixes=("_logged", "_modeled"))
ctr_by_article

,article_idx,impressions,clicks,ctr_observed,article_displayed_cnt,ctr_modeled
0,0,4963,103,0.020754,71.0,0.014085
1,1,4776,136,0.028476,20.0,0.000000
2,2,4962,173,0.034865,31.0,0.000000
3,3,4882,168,0.034412,30.0,0.033333
4,4,4829,116,0.024022,1792.0,0.039621
5,5,4771,183,0.038357,19.0,0.000000
6,6,4830,127,0.026294,33.0,0.000000
7,7,4996,98,0.019616,23.0,0.000000
8,8,4821,74,0.015350,29.0,0.000000
9,9,4744,114,0.024030,27.0,0.037037


In [ ]:
print(f'avg ctr_logged: {ctr_by_article["ctr_observed"].mean()}')
print(f'avg ctr_modeled: {ctr_by_article["ctr_modeled"].mean()}')

avg ctr_logged: 0.029106945409521717
avg ctr_modeled: 0.012242892721634566


In [ ]:
def ucb(alpha,mean, N, n_j):
  #mean: mean of the arm
  #N: number of total pulls
  #n_j: number of times the arm j has been pulled.
  return mean + np.sqrt(alpha*np.log(N) / n_j)

In [64]:
class Ucb1:
    def __init__(self, alpha):
        """
        Parameters
        ----------
        alpha : number (ucb parameter)
        """

        self.alpha = round(alpha, 1)
        self.algorithm = "UCB1 (α=" + str(self.alpha) + ")"

        self.q = np.zeros(n_arms)  # average reward for each arm
        self.n = np.ones(n_arms)  # number of times each arm was chosen

    
    def choose_arm(self, t, user, pool_idx):
        """
        Choose an arm to pull based on the UCB1 algorithm
        
        """
        # calculate the ucb for each arm
        ucbs = self.q[pool_idx] + np.sqrt(self.alpha * np.log(t+1)/self.n[pool_idx]) 
        # index of the arm with the highest ucb
        max_index = np.argmax(ucbs)
        return max_index

    def update(self, displayed, reward, user, pool_idx):
        """
        Updates algorithm's parameters(matrices)
        Parameters
        ----------
        displayed : index (displayed article index relative to the pool)
        reward : binary (user clicked or not)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        """
        # article or arm chosen by the algorithm
        a = displayed

        # update the number of times the arm was pulled
        self.n[a] += 1
        
        # update the average reward for the arm
        self.q[a] += (reward - self.q[a]) / self.n[a]


In [65]:
ucb_algo = Ucb1(.5)
_, deploy = evaluate(ucb_algo,learn_ratio=0.5)
rnd_ctr = deploy[-1]

UCB1 (α=0.5)        0.0215    1.6s


In [66]:
ucb_algo.q

array([0.04651163, 0.02702703, 0.02777778, 0.        , 0.01320132,
       0.02777778, 0.        , 0.        , 0.        , 0.02777778,
       0.        , 0.        , 0.02777778, 0.        , 0.06      ,
       0.02777778, 0.02777778, 0.06      , 0.02777778, 0.02777778,
       0.        ])

In [67]:
epsilon_analysis(ucb_algo, ctr_by_article)

avg ctr_logged: 0.029106945409521717
avg ctr_modeled: 0.020426771299439966


,article_idx,impressions,clicks,ctr_observed,article_displayed_cnt,ctr_modeled
0,0,4963,103,0.020754,43.0,0.046512
1,1,4776,136,0.028476,37.0,0.027027
2,2,4962,173,0.034865,36.0,0.027778
3,3,4882,168,0.034412,29.0,0.000000
4,4,4829,116,0.024022,1818.0,0.013201
5,5,4771,183,0.038357,36.0,0.027778
6,6,4830,127,0.026294,29.0,0.000000
7,7,4996,98,0.019616,29.0,0.000000
8,8,4821,74,0.015350,29.0,0.000000
9,9,4744,114,0.024030,36.0,0.027778


### Thomson Sampling

In [68]:
class thomsonSampling:
    def __init__(self, success, failure):
        """
        Parameters
        ----------
        alpha : number (ucb parameter)
        """


        self.algorithm = "Thomson (a=" + str(success) + ") (b=" + str(failure) + ")"

        self.a = np.ones(n_arms)*success  # average reward for each arm
        self.b = np.ones(n_arms)*failure # number of times each arm was pulled
        self.n = np.zeros(n_arms)  # number of times each arm was chosen

    
    def choose_arm(self, t, user, pool_idx):
        """
        Choose an arm to pull based on the UCB1 algorithm
        
        """
             
        thomson_sampling =[]
        for article_idx in pool_idx:
            a = self.a[article_idx] # a is the success rate of the arm
            b= self.b[article_idx] # b is the failure rate of the arm
            value = np.random.beta(a, b) # value is the probability of success of the the article(click)
            thomson_sampling.append(value)
        
        max_index = np.argmax(thomson_sampling) # we select the arm with the highest thomson sampling value (probability of success)
        return max_index 

    def update(self, displayed, reward, user, pool_idx):
        """
        Updates algorithm's parameters(matrices)
        Parameters
        ----------
        displayed : index (displayed article index relative to the pool)
        reward : binary (user clicked or not)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        """

        article_id = pool_idx[displayed]

        self.a[article_id] += reward  # update the success rate of the arm
        self.b[article_id] += 1-reward # update the failure rate of the arm
        self.n[article_id] += 1 # update the number of times the arm was chosen
        # update self.q[a] here

        

In [69]:
success, failures = 2,100

thomson_algo1 = thomsonSampling(1,100)
_, deploy1 = evaluate(thomson_algo1, learn_ratio=0.5)

thomson_algo2 = thomsonSampling(2, 100)
_, deploy2 = evaluate(thomson_algo2, learn_ratio=0.5)

thomson_algo3 = thomsonSampling(3, 100)
_, deploy3 = evaluate(thomson_algo3, learn_ratio=0.5)

thomson_algo4 = thomsonSampling(4, 100)
_, deploy4 = evaluate(thomson_algo4, learn_ratio=0.5)

thomson_algo5 = thomsonSampling(1,10)
_, deploy1 = evaluate(thomson_algo5, learn_ratio=0.5)

thomson_algo6 = thomsonSampling(.5,10)
_, deploy1 = evaluate(thomson_algo6, learn_ratio=0.5)

thomson_algo7 = thomsonSampling(1.5,10)
_, deploy1 = evaluate(thomson_algo7, learn_ratio=0.5)

Thomson (a=1) (b=100)0.0372    3.9s
Thomson (a=2) (b=100)0.03      2.9s
Thomson (a=3) (b=100)0.0308    2.9s
Thomson (a=4) (b=100)0.03      3.0s
Thomson (a=1) (b=10)0.0263    3.2s
Thomson (a=0.5) (b=10)0.0298    3.1s
Thomson (a=1.5) (b=10)0.0339    3.0s


In [70]:
def thopson_analysis(modeled_algo_instance, ctr_by_article):
    ctr_modeled_a = np.array(modeled_algo_instance.a) # a is the success rate of the arm
    ctr_modeled_b = np.array(modeled_algo_instance.b) # b is the failure rate of the arm
    ctr_modeled = ctr_modeled_a / (ctr_modeled_a + ctr_modeled_b) # expected mean of the CTR of the modeled algorithm since its a beta distribution
    #expected_mean_reward = ctr_modeled_a / (ctr_modeled_b + ctr_modeled_a)
    article_displayed = modeled_algo_instance.n # number of times each arm was chosen
    epsilon_algo_data = pd.DataFrame({"article_displayed_cnt": article_displayed, "ctr_modeled": ctr_modeled}).sort_values(by="ctr_modeled", ascending=False).reset_index()
    epsilon_algo_data.rename(columns={"index": "article_idx"}, inplace=True)
    ctr_by_article = ctr_by_article.merge(epsilon_algo_data, on="article_idx", suffixes=("_logged", "_modeled"))
    print(f'avg ctr_logged: {ctr_by_article["ctr_observed"].mean()}')
    print(f'avg ctr_modeled: {ctr_by_article["ctr_modeled"].mean()}')
    return ctr_by_article

In [78]:
thopson_analysis(thomson_algo4, ctr_by_article)

avg ctr_logged: 0.029106945409521717
avg ctr_modeled: 0.03198216356594184


,article_idx,impressions,clicks,ctr_observed,article_displayed_cnt,ctr_modeled
0,0,4963,103,0.020754,77.0,0.027624
1,1,4776,136,0.028476,86.0,0.031579
2,2,4962,173,0.034865,66.0,0.023529
3,3,4882,168,0.034412,55.0,0.025157
4,4,4829,116,0.024022,187.0,0.037801
5,5,4771,183,0.038357,316.0,0.040476
6,6,4830,127,0.026294,147.0,0.039841
7,7,4996,98,0.019616,65.0,0.029586
8,8,4821,74,0.015350,66.0,0.029412
9,9,4744,114,0.024030,192.0,0.033784
